In [1]:
import sys
sys.path.append('../..')

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import feature_column as fc
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import optimizers
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import shuffle
from utils.gpu.cuda_cluster import *
from utils.gpu.preprocessing import read_data, factorize_small_cardinality_with_index, df_to_tfdataset, split_join, get_media_index
from utils.evaluate import calculate_ctr, compute_rce, average_precision_score
import core.config as conf

/home/nyongja/anaconda3/envs/dask/lib/python3.7/site-packages/distributed/node.py:155: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 46821 instead
  http_address["port"], self.http_server.port


In [2]:
client

Client Scheduler: tcp://127.0.0.1:40109 Dashboard: http://127.0.0.1:46821/status,Cluster Workers: 1 Cores: 1 Memory: 33.47 GB


## Load data

In [3]:
# data_path = conf.raw_data_path + '*' # for all dataset
data_path = conf.data_root + 'dask_input'
df = dask_cudf.read_parquet(f'{data_path}/train-final-te-retweet_comment-1.parquet', index=False)

## Preprocessing

In [4]:
df.head()

,timestamp,creator_follower_count,creator_following_count,creator_is_verified,creator_account_creation,engager_follower_count,engager_following_count,engager_is_verified,engager_account_creation,engager_follows_creator,...,TE_creator_count_combined_tweet_type_language_retweet_comment,TE_creator_user_fer_count_delta_time_media_language_retweet_comment,TE_creator_user_fing_count_delta_time_media_language_retweet_comment,TE_creator_user_fering_count_delta_time_tweet_type_language_retweet_comment,TE_creator_user_fing_count_mode_media_language_retweet_comment,TE_creator_user_fer_count_mode_media_language_retweet_comment,TE_creator_user_fering_count_mode_tweet_type_language_retweet_comment,TE_domains_media_tweet_type_language_retweet_comment,TE_links_media_tweet_type_language_retweet_comment,TE_hashtags_media_tweet_type_language_retweet_comment
0,1612772154,389,938,0,1461219816,496,100,0,1447563660,1,...,0.007486,0.014033,0.014033,0.007486,0.013307,0.013307,0.007150,0.011783,0.011783421,0.013712
1,1614050312,17069714,189,1,1344977596,8,17,0,1546057245,0,...,0.006329,0.007339,0.007339,0.006329,0.007291,0.007291,0.006284,0.005441,0.005441219,0.005482
2,1613017908,86963,230,0,1495978710,351,314,0,1480829656,0,...,0.006126,0.007410,0.007410,0.006126,0.007375,0.007375,0.006009,0.006031,0.00603065,0.006208
3,1613668747,27544,2771,1,1270641352,284,1422,0,1231503632,0,...,0.008626,0.006266,0.006266,0.008626,0.006266,0.006266,0.008626,0.003687,<NA>,0.004379
4,1612495815,186,177,0,1591281565,423,49,0,1554115261,1,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000


In [10]:
df['TE_creator_user_fer_count_mode_media_language_retweet_comment'].isnull().sum().compute()

146016

In [14]:
df[df['TE_creator_user_fer_count_mode_media_language_retweet_comment'].isnull()].compute()

,timestamp,creator_follower_count,creator_following_count,creator_is_verified,creator_account_creation,engager_follower_count,engager_following_count,engager_is_verified,engager_account_creation,engager_follows_creator,...,TE_creator_count_combined_tweet_type_language_retweet_comment,TE_creator_user_fer_count_delta_time_media_language_retweet_comment,TE_creator_user_fing_count_delta_time_media_language_retweet_comment,TE_creator_user_fering_count_delta_time_tweet_type_language_retweet_comment,TE_creator_user_fing_count_mode_media_language_retweet_comment,TE_creator_user_fer_count_mode_media_language_retweet_comment,TE_creator_user_fering_count_mode_tweet_type_language_retweet_comment,TE_domains_media_tweet_type_language_retweet_comment,TE_links_media_tweet_type_language_retweet_comment,TE_hashtags_media_tweet_type_language_retweet_comment
12,1613132699,6752,5333,0,1293568586,270,326,0,1286915405,1,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0.008975592,0.008975592,0.008913431
126,1613112622,246,444,0,1597062356,716,844,0,1580745826,1,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0.002590481,0.002590481,0.002698955
177,1613097466,278107,582,1,1243810764,41,219,0,1580740501,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0.00746042,0.00746042,0.007637406
225,1613130614,105436,585,1,1232991786,30,125,0,1412805857,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0.017434383,<NA>,0.008913431
228,1613144349,71441,275,0,1433061291,93,551,0,1524199160,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0.008141845,0.008141845,0.009205014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3078900,1613095178,4598657,522,1,1294682107,114,241,0,1524186657,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0.010041894,0.010041894,0.00051491
3078944,1613126784,993,1787,0,1561226477,63,113,0,1560967764,1,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0.010043914,0.010043914,0.009492236
3079014,1613147263,63816,446,0,1517021808,222,744,0,1592561986,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0.003118313,0.003118313,0.002743704
3079019,1613170799,33262,346,0,1581159223,187,652,0,1253634556,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0.008307333,0.008307333,0.008953992


In [5]:
df.isnull().sum().compute()

timestamp                                                                     0
creator_follower_count                                                        0
creator_following_count                                                       0
creator_is_verified                                                           0
creator_account_creation                                                      0
                                                                          ...  
TE_creator_user_fer_count_mode_media_language_retweet_comment            146016
TE_creator_user_fering_count_mode_tweet_type_language_retweet_comment    145772
TE_domains_media_tweet_type_language_retweet_comment                      78933
TE_links_media_tweet_type_language_retweet_comment                       306753
TE_hashtags_media_tweet_type_language_retweet_comment                    340103
Length: 72, dtype: uint64

In [6]:
df = df.fillna(0)

In [7]:
df = df.astype(np.int64)

## Sampling

In [8]:
df_positive = df[df['retweet_comment']==1]

In [9]:
df_negative = df[df['retweet_comment']==0]

In [10]:
print(len(df_positive))
print(len(df_negative))

21659
3057388


In [11]:
df_negative = df_negative.compute().sample(n = len(df_positive))

In [12]:
df = dask.dataframe.concat([df_positive, df_negative])

In [19]:
df = shuffle(df)

In [21]:
df = df.reset_index(drop=True)

In [22]:
df

,timestamp,creator_follower_count,creator_following_count,creator_is_verified,creator_account_creation,engager_follower_count,engager_following_count,engager_is_verified,engager_account_creation,engager_follows_creator,...,TE_creator_count_combined_tweet_type_language_retweet_comment,TE_creator_user_fer_count_delta_time_media_language_retweet_comment,TE_creator_user_fing_count_delta_time_media_language_retweet_comment,TE_creator_user_fering_count_delta_time_tweet_type_language_retweet_comment,TE_creator_user_fing_count_mode_media_language_retweet_comment,TE_creator_user_fer_count_mode_media_language_retweet_comment,TE_creator_user_fering_count_mode_tweet_type_language_retweet_comment,TE_domains_media_tweet_type_language_retweet_comment,TE_links_media_tweet_type_language_retweet_comment,TE_hashtags_media_tweet_type_language_retweet_comment
0,1612669305,94,75,0,1590843281,181,79,0,1599157792,1,...,0,0,0,0,0,0,0,0,0,0
1,1614093537,1280,1721,0,1562364076,212,762,0,1314813239,1,...,0,0,0,0,0,0,0,0,0,0
2,1613345508,122,195,0,1526409266,299,222,0,1424345742,1,...,0,0,0,0,0,0,0,0,0,0
3,1612571206,231,243,0,1597512665,141,191,0,1581032968,1,...,0,0,0,0,0,0,0,0,0,0
4,1613530451,3274,3652,0,1422139854,1257,798,0,1412892752,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43313,1613273557,532,317,0,1275592460,76,105,0,1419803241,1,...,0,0,0,0,0,0,0,0,0,0
43314,1614004394,265,513,0,1356814616,394,175,0,1427986296,1,...,0,0,0,0,0,0,0,0,0,0
43315,1612566482,313024,3440,0,1447814497,6,21,0,1594239434,0,...,0,0,0,0,0,0,0,0,0,0
43316,1612887309,5251,5197,0,1341629891,64,35,0,1307518179,1,...,0,0,0,0,0,0,0,0,0,0


## Scaling

In [20]:
df_y = df[['retweet_comment']]
df_x = df.drop(['retweet_comment'], axis = 1)

In [21]:
X_train = df_x
y_train = df_y['retweet_comment'].to_frame()

In [25]:
mms = MinMaxScaler(feature_range = (0, 1))
X_train = mms.fit_transform(X_train.as_matrix())

In [27]:
X_train = pd.DataFrame(X_train, columns = df_x.columns)

In [20]:
standard_scaler = preprocessing.StandardScaler()
standard_scaler.fit(X_train.as_matrix())

StandardScaler()

In [21]:
ss = standard_scaler.transform(X_train.as_matrix())

In [22]:
X_train = pd.DataFrame(ss, columns = df_x.columns)

## Split data

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2)

In [30]:
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.2)

## Modeling

In [38]:
model = Sequential([
    Dense(64, activation = 'relu', input_dim = X_test.shape[1]),
    Dense(32, activation = 'relu'),
    Dense(16, activation = 'relu'),
    Dense(1, activation = 'sigmoid')
])

In [39]:
model.compile(
    optimizer = optimizers.Adam(learning_rate = 0.07),
    loss = 'binary_crossentropy', # softmax : sparse_categorical_crossentropy, sigmoid : binary_crossentropy
    metrics=['accuracy']
    )

In [40]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 64)                4608      
_________________________________________________________________
dense_5 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_6 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 17        
Total params: 7,233
Trainable params: 7,233
Non-trainable params: 0
_________________________________________________________________


In [41]:
model.fit(
    x = X_train,
    y = y_train.to_pandas(),
    validation_data=(X_val, y_val.to_pandas()),
    epochs=5,
    batch_size=64
)

Epoch 1/5
542/542 [==============================] - 1s 602us/step - loss: 0.3789 - accuracy: 0.8392 - val_loss: 0.1983 - val_accuracy: 0.9348
Epoch 2/5
542/542 [==============================] - 0s 455us/step - loss: 0.1920 - accuracy: 0.9299 - val_loss: 0.3191 - val_accuracy: 0.8736
Epoch 3/5
542/542 [==============================] - 0s 460us/step - loss: 0.2579 - accuracy: 0.9076 - val_loss: 0.2364 - val_accuracy: 0.9169
Epoch 4/5
542/542 [==============================] - 0s 466us/step - loss: 0.2465 - accuracy: 0.9132 - val_loss: 0.2556 - val_accuracy: 0.9059
Epoch 5/5
542/542 [==============================] - 0s 459us/step - loss: 0.2140 - accuracy: 0.9288 - val_loss: 0.1873 - val_accuracy: 0.9354


In [42]:
model.evaluate(X_test, y_test.to_pandas())

217/217 [==============================] - 0s 281us/step - loss: 0.1845 - accuracy: 0.9364


[0.18445414304733276, 0.9363728165626526]

## Predict

In [43]:
predict_re_comment = model.predict(X_test)

In [44]:
predict_re_comment

array([[0.07833111],
       [1.        ],
       [0.07833111],
       ...,
       [0.07833111],
       [0.07833111],
       [0.07833111]], dtype=float32)

In [54]:
y_test['predict'] = predict_re_comment.tolist()

In [56]:
rce_like = compute_rce(predict_re_comment, y_test['retweet_comment'].to_array())
rce_like

/home/nyongja/anaconda3/envs/dask/lib/python3.7/site-packages/sklearn/metrics/_classification.py:2279: RuntimeWarning: divide by zero encountered in log
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
/home/nyongja/anaconda3/envs/dask/lib/python3.7/site-packages/sklearn/metrics/_classification.py:2279: RuntimeWarning: invalid value encountered in multiply
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)


nan

In [50]:
average_precision_score(y_test['retweet_comment'].to_array(), predict_re_comment)

0.9633902348089741